# Robust estimation of Tully-Fisher extragalactic distance errors in NED-D

Companion notebook for the "Predicting extragalactic distance errors using bayesian inference in multi measurement catalogues" paper, Monthly Notices of the Royal Astronomical Society, Volume 485, Issue 3, May 2019, Pages 4343–4358, https://doi.org/10.1093/mnras/stz615

This notebook shows:

- A bootstrap robust estimation of TFR distance errors (H,M) and frequentist distance errors (P,Q)

The following notebook is [here](bayesian_models.ipynb).

In [ ]:
%%bash
wget https://github.com/saint-germain/anisotropias/raw/master/NED28.10.1-D-15.1.0-20181130.csv

In [1]:
import pandas as pd
import numpy as np
import collections
pd.options.mode.chained_assignment = None

In [2]:
def weighted_std(values, weights):
    """
    Return the weighted average and standard deviation.
    Richard M. Brugger, "A Note on Unbiased Estimation of the Standard Deviation", The American Statistician (23) 4 p. 32 (1969)

    values, weights -- Numpy ndarrays with the same shape.
    """
    n=len(values)
    average = np.average(values, weights=weights)
    variance = (n/(n-1.5))*np.average((values-average)**2, weights=weights)  # Fast and numerically precise
    return np.sqrt(variance),average

In [3]:
df=pd.read_csv("NED28.10.1-D-15.1.0-20181130.csv",skiprows=12)
df=df[np.isnan(df['redshift (z)'])] # only measurements without redshift data are useful here

In [4]:
def selectdata_lite(mymethod,df):
    dfa=df[~np.isfinite(df.err)|(df.err==0)] # database of non reported errors
    df1=df[np.isfinite(df.err)] # remove measurements that do not report an error
    df1=df1[df1.err!=0] # remove measurements that report an error as zero ¬¬
# Select a method for analysis
    filter=np.full(len(df1), False, dtype=bool)
    for mym in mymethod:
        filter|=(df1.Method==mym) # choose  methods
    df1=df1[filter] 
    namelist=list(df1['Galaxy ID']) # list of galaxies
    counter=collections.Counter(namelist) # count measurements per galaxy
# Select galaxies with a minimum number of measurements
    ulist=[]
    ulist2=[]
    nmeas=1
    for i in counter.keys():
        if counter[i]>nmeas:
            ulist+=[i] # all galaxies with more than n_meas measurements
        if counter[i]>=1:
            ulist2+=[i] # all galaxies with at least one measurement w/a reported error
    print('No. of Galaxies with reported errors is %i' % len(ulist2) )
    print('No. of Galaxies with more than %i measurements is %i' % (nmeas,len(ulist)) )
# Create database for bootstrap, remove unnecessary columns
    dfs=df1[np.in1d(df1['Galaxy ID'],ulist)] # dataframe with galaxies with more than nmeas measurements
    colu=list(df.columns)
    for i in ['Galaxy ID', 'm-M', 'err', 'D (Mpc)']:
        colu.remove(i)
    dfs.drop(colu, inplace=True, axis=1)
# Create database for non-reported errors
    filter=np.full(len(dfa), False, dtype=bool)
    for mym in mymethod:
        filter|=(dfa.Method==mym) # choose  methods
    df1a=dfa[filter] 
    elist=list(np.unique(df1a['Galaxy ID'])) # list of galaxies with measurements without reported errors
    filtr=~np.in1d(elist,ulist2) # select only galaxies with no additional measurements using the same method
    nulista=np.asarray(elist)[filtr] # list of galaxies without reported errors
    print('No. of Galaxies without reported errors is %i' % len(nulista) )
    return ulist,dfs,nulista,df1a,len(nulista),len(ulist),len(ulist2)

In [5]:
mymethod=['Tully-Fisher','Comp. Secondary: Tully-Fisher']
ulist,dfs,nulista,odf,*mma=selectdata_lite(mymethod,df)

No. of Galaxies with reported errors is 11408
No. of Galaxies with more than 1 measurements is 9114
No. of Galaxies without reported errors is 884


In [6]:
odf=odf[np.in1d(odf['Galaxy ID'], nulista)]
odf.to_csv("unreportedTF.csv")

Only recalculate the following if necessary

In [ ]:
%%time
np.random.seed(10)
nbins=10000 # 1e4 -> 4 minutes
em=[] # number of measurements per galaxy
bootp50=[] # mean error from the bootstrap for each galaxy - H error
bootmsig=[] # variance of M error
bootmother=[] # mad-variance of M error
bootsig=[] # variance of H error
dboot=[] # mean bootstrap distance
bootmad=[] # median absolute deviation - M error
wsnt=[] # error propagated from the weighted standard deviation of the distance modulus 
dwa=[] # distance using weighted average of modulus and err (for show mostly)
ecf=[] # propagated (cosmicflows) error - P error
eqd=[] # quadrature sum of P error and weighted stdev - Q error
for i in ulist:
    dfilter=np.in1d(dfs['Galaxy ID'],i)
    dummy=dfs[dfilter]
    em+=[len(dummy)] # n_meas, number of measurements per galaxy
    tli=[]
    for km,ke in zip(dummy['m-M'],dummy['err']):
        tli+=[list(10**(np.random.normal(km,ke,nbins)/5.+1))] # bootstrap draw for each gal
    tli=np.array(tli)
    booterr=(np.percentile(tli, 84,axis=0)-np.percentile(tli, 16,axis=0))/2 # sigma draws from bootstrap for each gal
    bootmean=np.median(tli,axis=0) # mean draws from bootstrap for each gal
############## This block should run for 10k draws ###############
#    mymed=np.median(tli,axis=0) #median for error of m error
#    mst=[np.median(np.abs(tli[:,kk]-mymed[kk])) for kk in range(nbins)]
#    bootmsig+=[(np.percentile(mst, 84,axis=0)-np.percentile(mst, 16,axis=0))/2e6] # error of m error
#    bootmother+=[np.median(mst)/1e6]
###################################################################
    bootp50+=[np.median(booterr)/1e6] 
    bootsig+=[((np.percentile(booterr, 84)-np.percentile(booterr, 16)))/2e6] 
    bootmad+=[np.median(np.abs(tli - np.median(tli)))/1e6]
    dboot+=[np.median(bootmean)/1e6] 
    wnat,avnat = weighted_std(dummy['m-M'],1/dummy['err']**2)
    distwav=10**(avnat/5+1)/1e6
    dwa+=[distwav] #
    wsti=0.461*distwav*wnat # propagated weighted standard deviation
    wsnt+=[wsti] 
    ecfi=0.461*distwav/np.sqrt((1/dummy['err']**2).sum()) 
    ecf+=[ecfi]
    eqd+=[np.sqrt(ecfi**2+wsti**2)]    
# weighted std using D and propagated err is similar to propagated D error using weighted std on modulus and error

### Do not re-write bootstrap results by executing the next couple of cells. For emergencies only!

In [ ]:
d = {'names': ulist, 'meas': em, 'bootp50':bootp50, 'bootsig':bootsig,'dboot':dboot,'wsnt':wsnt,'dwa':dwa,'ecf':ecf,'eqd':eqd,'bootmad':bootmad}
dfb = pd.DataFrame(data=d)
dfb.to_csv("bootstrap_results_2018.csv")

In [ ]:
d = {'names': ulist, 'meas': em, 'bootmad':bootmad,'bootmsig':bootmsig,'bootmother':bootmother}
dfm = pd.DataFrame(data=d)
dfm.to_csv("bootstrap_results_wm_2018.csv")

In [ ]:
# the above are already in an online github repository
# https://github.com/saint-germain/anisotropias/raw/master/bootstrap_results_2018.csv
# https://github.com/saint-germain/anisotropias/raw/master/bootstrap_results_wm_2018.csv